In [1]:
import pandas as pd
import numpy as np
import tsfresh
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import tensorflow as tf

import os

import matplotlib.pyplot as plt

# keras goodies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, Dropout, MaxPooling1D, BatchNormalization, LSTM
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import metrics as kmetrics
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LeakyReLU

In [2]:
base_df_2021_2022 = pd.read_csv('2021_2022_clean_data.csv')

C:\Users\jacks\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3135: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
# only requires four type, others delete.

In [3]:
columns_of_interest = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']

In [4]:
window_size = 25 # 50 datapoints for the window size, which, at 25Hz, means 2 seconds
step_size = 5 # this is 50% overlap

window_number = 0 # start a counter at 0 to keep track of the window number

all_overlapping_windows = []

for rid, group in base_df_2021_2022.groupby("recording_id"):
    print(f"Processing rid = {rid}")
    
    large_enough_windows = [window for window in group.rolling(window=window_size, min_periods=window_size) if len(window) == window_size]
    
    overlapping_windows = large_enough_windows[::step_size] 
    
    # then we will append a window ID to each window
    for window in overlapping_windows:
        window.loc[:, 'window_id'] = window_number
        window_number += 1
    
    all_overlapping_windows.append(pd.concat(overlapping_windows).reset_index(drop=True))

Processing rid = C:\2022\iot\our_branch\Labs\Data\2022_A\day1\Respeck_s1810150_Standing_06-10-2022_20-49-00
Processing rid = Respeck_s1541031_Climbing stairs_06-10-2021_11-25-43


C:\Users\jacks\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\jacks\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Processing rid = Respeck_s1541031_Descending stairs_06-10-2021_11-25-05
Processing rid = Respeck_s1541031_Desk work_06-10-2021_11-04-49
Processing rid = Respeck_s1541031_Lying down left_06-10-2021_14-30-04
Processing rid = Respeck_s1541031_Lying down on back_06-10-2021_11-15-13
Processing rid = Respeck_s1541031_Lying down on stomach_06-10-2021_14-30-50
Processing rid = Respeck_s1541031_Lying down right_06-10-2021_14-27-40
Processing rid = Respeck_s1541031_Movement_06-10-2021_11-11-18
Processing rid = Respeck_s1541031_Running_08-10-2021_12-10-37
Processing rid = Respeck_s1541031_Sitting bent backward_06-10-2021_14-33-45
Processing rid = Respeck_s1541031_Sitting bent forward_06-10-2021_14-32-55
Processing rid = Respeck_s1541031_Sitting_06-10-2021_10-25-01
Processing rid = Respeck_s1541031_Standing_06-10-2021_11-10-10
Processing rid = Respeck_s1541031_Walking at normal speed_06-10-2021_18-30-14
Processing rid = Respeck_s1704145_Climbing stairs_23-09-2021_11-34-34
Processing rid = Respeck_

Processing rid = Respeck_s1732873_Sitting_04-10-2021_12-53-07
Processing rid = Respeck_s1732873_Sitting_04-10-2022_14-07-59
Processing rid = Respeck_s1732873_Standing_04-10-2021_12-55-45
Processing rid = Respeck_s1732873_Standing_04-10-2022_14-10-08
Processing rid = Respeck_s1732873_Walking at normal speed_04-10-2021_15-30-06
Processing rid = Respeck_s1732873_Walking at normal speed_04-10-2022_14-31-37
Processing rid = Respeck_s1757177_Climbing stairs_06-10-2021_16-34-18
Processing rid = Respeck_s1757177_Descending stairs_02-10-2021_13-57-31
Processing rid = Respeck_s1757177_Desk work_02-10-2021_13-47-55
Processing rid = Respeck_s1757177_Lying down left_06-10-2021_20-37-42
Processing rid = Respeck_s1757177_Lying down on back_02-10-2021_19-55-43
Processing rid = Respeck_s1757177_Lying down on stomach_02-10-2021_20-00-57
Processing rid = Respeck_s1757177_Lying down right_06-10-2021_20-38-38
Processing rid = Respeck_s1757177_Movement_06-10-2021_16-44-16
Processing rid = Respeck_s1757177_R

Processing rid = Respeck_s1817967_Lying down left_30-09-2022_16-25-54
Processing rid = Respeck_s1817967_Lying down on back_30-09-2022_16-24-43
Processing rid = Respeck_s1817967_Lying down on stomach_30-09-2022_16-27-33
Processing rid = Respeck_s1817967_Lying down right_30-09-2022_16-26-44
Processing rid = Respeck_s1817967_Movement_30-09-2022_16-35-29
Processing rid = Respeck_s1817967_Running_30-09-2022_16-30-26
Processing rid = Respeck_s1817967_Sitting bent backward_30-09-2022_16-21-38
Processing rid = Respeck_s1817967_Sitting bent forward_30-09-2022_16-23-28
Processing rid = Respeck_s1817967_Sitting_30-09-2022_16-19-45
Processing rid = Respeck_s1817967_Standing_30-09-2022_16-22-44
Processing rid = Respeck_s1817967_Walking at normal speed_30-09-2022_16-29-44
Processing rid = Respeck_s1817972_Climbing stairs_05-10-2021_14-15-10
Processing rid = Respeck_s1817972_Descending stairs_05-10-2021_14-14-27
Processing rid = Respeck_s1817972_Desk work_05-10-2021_11-25-28
Processing rid = Respeck_

...

Processing rid = Respeck_s1948094_Descending stairs_23-09-2022_12-53-24
Processing rid = Respeck_s1948094_Desk work_23-09-2022_12-01-53
Processing rid = Respeck_s1948094_Lying down left_23-09-2022_11-54-37
Processing rid = Respeck_s1948094_Lying down on back_23-09-2022_11-59-08
Processing rid = Respeck_s1948094_Lying down on stomach_23-09-2022_11-57-49
Processing rid = Respeck_s1948094_Lying down right_23-09-2022_11-56-22
Processing rid = Respeck_s1948094_Movement_23-09-2022_12-58-07
Processing rid = Respeck_s1948094_Running_23-09-2022_12-54-29
Processing rid = Respeck_s1948094_Sitting bent backward_23-09-2022_11-50-08
Processing rid = Respeck_s1948094_Sitting bent forward_23-09-2022_11-48-20
Processing rid = Respeck_s1948094_Sitting_23-09-2022_11-45-44
Processing rid = Respeck_s1948094_Standing_23-09-2022_11-51-34
Processing rid = Respeck_s1948094_Walking at normal speed_23-09-2022_12-49-50
Processing rid = Respeck_s1951693_Climbing stairs_05-10-2022_16-29-06
Processing rid = Respeck_

Processing rid = Respeck_s1962038_Running_23-09-2022_15-44-56
Processing rid = Respeck_s1962038_Sitting bent backward_04-10-2022_14-47-08
Processing rid = Respeck_s1962038_Sitting bent forward_23-09-2022_14-56-59
Processing rid = Respeck_s1962038_Sitting_23-09-2022_14-54-11
Processing rid = Respeck_s1962038_Standing_04-10-2022_14-45-21
Processing rid = Respeck_s1962038_Walking at normal speed_02-10-2022_19-03-08
Processing rid = Respeck_s1965695_Climbing stairs_29-09-2022_17-32-23
Processing rid = Respeck_s1965695_Descending stairs_29-09-2022_17-31-42
Processing rid = Respeck_s1965695_Desk work_29-09-2022_13-43-35
Processing rid = Respeck_s1965695_Lying down left_29-09-2022_13-34-41
Processing rid = Respeck_s1965695_Lying down on back_29-09-2022_13-32-30
Processing rid = Respeck_s1965695_Lying down on stomach_29-09-2022_13-37-53
Processing rid = Respeck_s1965695_Lying down right_29-09-2022_13-35-36
Processing rid = Respeck_s1965695_Movement_29-09-2022_13-40-59
Processing rid = Respeck_

Processing rid = Respeck_s1977764_Desk work_24-09-2022_15-51-41
Processing rid = Respeck_s1977764_Lying down left_24-09-2022_15-42-19
Processing rid = Respeck_s1977764_Lying down on back_24-09-2022_15-41-30
Processing rid = Respeck_s1977764_Lying down on stomach_24-09-2022_15-44-05
Processing rid = Respeck_s1977764_Lying down right_24-09-2022_15-43-10
Processing rid = Respeck_s1977764_Movement_24-09-2022_15-50-36
Processing rid = Respeck_s1977764_Running_24-09-2022_15-46-23
Processing rid = Respeck_s1977764_Sitting bent backward_24-09-2022_15-38-53
Processing rid = Respeck_s1977764_Sitting bent forward_24-09-2022_15-37-51
Processing rid = Respeck_s1977764_Sitting_24-09-2022_15-36-44
Processing rid = Respeck_s1977764_Standing_24-09-2022_15-39-50
Processing rid = Respeck_s1977764_Walking at normal speed_24-09-2022_15-45-27
Processing rid = Respeck_s1992413_Climbing stairs_29-09-2022_17-49-07
Processing rid = Respeck_s1992413_Descending stairs_29-09-2022_17-48-20
Processing rid = Respeck_

Processing rid = Respeck_s2181154_Running_29-09-2021_14-01-11
Processing rid = Respeck_s2181154_Sitting bent backward_29-09-2021_14-10-58
Processing rid = Respeck_s2181154_Sitting bent forward_29-09-2021_14-09-26
Processing rid = Respeck_s2181154_Sitting_29-09-2021_13-58-39
Processing rid = Respeck_s2181154_Standing_03-10-2021_17-44-24
Processing rid = Respeck_s2181154_Walking at normal speed_29-09-2021_14-00-32
Processing rid = Respeck_s2192970_Climbing stairs_29-09-2021_21-39-04
Processing rid = Respeck_s2192970_Descending stairs_29-09-2021_21-39-49
Processing rid = Respeck_s2192970_Desk work_29-09-2021_20-24-25
Processing rid = Respeck_s2192970_Lying down left_29-09-2021_20-26-47
Processing rid = Respeck_s2192970_Lying down on back_29-09-2021_20-26-05
Processing rid = Respeck_s2192970_Lying down on stomach_29-09-2021_20-28-13
Processing rid = Respeck_s2192970_Lying down right_29-09-2021_20-27-26
Processing rid = Respeck_s2192970_Movement_29-09-2021_21-36-55
Processing rid = Respeck_

Processing rid = Respeck_s2299270_Lying down left_03-10-2022_16-00-31
Processing rid = Respeck_s2299270_Lying down on back_03-10-2022_15-59-43
Processing rid = Respeck_s2299270_Lying down on stomach_03-10-2022_16-04-09
Processing rid = Respeck_s2299270_Lying down right_03-10-2022_16-02-21
Processing rid = Respeck_s2299270_Movement_03-10-2022_17-24-56
Processing rid = Respeck_s2299270_Running_30-09-2022_17-00-24
Processing rid = Respeck_s2299270_Sitting bent backward_03-10-2022_15-56-25
Processing rid = Respeck_s2299270_Sitting bent forward_03-10-2022_15-54-41
Processing rid = Respeck_s2299270_Sitting_03-10-2022_12-06-19
Processing rid = Respeck_s2299270_Standing_03-10-2022_11-08-12
Processing rid = Respeck_s2299270_Walking at normal speed_03-10-2022_16-53-10
Processing rid = Respeck_s2303353_Climbing stairs_22-09-2022_14-32-51
Processing rid = Respeck_s2303353_Descending stairs_22-09-2022_14-34-06
Processing rid = Respeck_s2303353_Desk work_22-09-2022_14-29-24
Processing rid = Respeck_

In [5]:
# Sitting/Standing, Walking, Lying Down, Running
final_sliding_windows = pd.concat(all_overlapping_windows).reset_index(drop=True)
final_sliding_windows.groupby("window_id")[['activity_type']].agg(np.min)
class_labels = {
    'Sitting':0,
    'Walking at normal speed': 1,
    'Lying down on back': 3,
    'Sitting bent forward':0,
    'Sitting bent backward':0,
    'Lying down right':3,
    'Lying down left':3,
    'Lying down on stomach':3,
    'Standing':0,
    'Running':2, 
}
window_id_class_labels = final_sliding_windows.groupby("window_id")[['activity_type']].agg(np.min).replace(class_labels)

In [7]:
X = []
y = []

for window_id, group in final_sliding_windows.groupby('window_id'):
    print(f"window_id = {window_id}")
    
    shape = group[columns_of_interest].values.shape
    print(f"shape = {shape}")
    
    X.append(group[columns_of_interest].values)
    y.append(class_labels[group["activity_type"].values[0]])
    
X = np.asarray(X)
y = np.asarray(y)

print(f"X shape = {X.shape}")
print(f"y shape = {y.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, train_size=0.8)

y_train = np.asarray(pd.get_dummies(y_train), dtype=np.float32)
y_test = np.asarray(pd.get_dummies(y_test), dtype=np.float32)
print(f"X_train shape = {X_train.shape}")
print(f"y_train shape = {y_train.shape}")

print(f"X_test shape = {X_test.shape}")
print(f"y_test shape = {y_test.shape}")

window_id = 0
shape = (25, 6)
window_id = 1
shape = (25, 6)
window_id = 2
shape = (25, 6)
window_id = 3
shape = (25, 6)
window_id = 4
shape = (25, 6)
window_id = 5
shape = (25, 6)
window_id = 6
shape = (25, 6)
window_id = 7
shape = (25, 6)
window_id = 8
shape = (25, 6)
window_id = 9
shape = (25, 6)
window_id = 10
shape = (25, 6)
window_id = 11
shape = (25, 6)
window_id = 12
shape = (25, 6)
window_id = 13
shape = (25, 6)
window_id = 14
shape = (25, 6)
window_id = 15
shape = (25, 6)
window_id = 16
shape = (25, 6)
window_id = 17
shape = (25, 6)
window_id = 18
shape = (25, 6)
window_id = 19
shape = (25, 6)
window_id = 20
shape = (25, 6)
window_id = 21
shape = (25, 6)
window_id = 22
shape = (25, 6)
window_id = 23
shape = (25, 6)
window_id = 24
shape = (25, 6)
window_id = 25
shape = (25, 6)
window_id = 26
shape = (25, 6)
window_id = 27
shape = (25, 6)
window_id = 28
shape = (25, 6)
window_id = 29
shape = (25, 6)
window_id = 30
shape = (25, 6)
window_id = 31
shape = (25, 6)
window_id = 32
sha

KeyError: 'Climbing stairs'

In [8]:
filters = 128
kernel_size = 4
n_features = 6
activation='relu'
n_classes = 4

In [9]:
# time-base learning rate decay
# 0.97 accuracy
epochs = 100
learning_rate = 0.001
decay_rate = learning_rate/epochs
model = Sequential()

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                 input_shape=(window_size, n_features)))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(MaxPooling1D(pool_size=3,strides=1,padding="same"))

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(MaxPooling1D(pool_size=3,strides=1,padding="same"))

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear'))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(MaxPooling1D(pool_size=3,strides=1,padding="same"))

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear'))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(MaxPooling1D(pool_size=3,strides=1,padding="same"))

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear'))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(MaxPooling1D(pool_size=3,strides=1,padding="same"))


model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

# early_stopping = EarlyStopping(monitor='val_loss',patience=10)
model.compile(
    optimizer=optimizers.Adam(learning_rate=learning_rate,decay =decay_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy', 'Recall', 'Precision'])
history_reg = model.fit(X_train, y_train, batch_size=64, epochs=epochs, validation_split=0.1)

print(history_reg.history)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 22, 128)           3200      
_________________________________________________________________
batch_normalization (BatchNo (None, 22, 128)           512       
_________________________________________________________________
activation (Activation)      (None, 22, 128)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 22, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 19, 128)           65664     
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 128)           512       
_________________________________________________________________
activation_1 (Activation)    (None, 19, 128)           0

Epoch 56/100
1955/1955 [==============================] - 10s 5ms/step - loss: 0.0224 - accuracy: 0.9946 - recall: 0.9946 - precision: 0.9946 - val_loss: 0.0196 - val_accuracy: 0.9960 - val_recall: 0.9960 - val_precision: 0.9960
Epoch 57/100
1955/1955 [==============================] - 10s 5ms/step - loss: 0.0214 - accuracy: 0.9949 - recall: 0.9949 - precision: 0.9949 - val_loss: 0.0204 - val_accuracy: 0.9949 - val_recall: 0.9948 - val_precision: 0.9949
Epoch 58/100
1955/1955 [==============================] - 10s 5ms/step - loss: 0.0210 - accuracy: 0.9954 - recall: 0.9954 - precision: 0.9954 - val_loss: 0.0187 - val_accuracy: 0.9961 - val_recall: 0.9961 - val_precision: 0.9961
Epoch 59/100
1955/1955 [==============================] - 10s 5ms/step - loss: 0.0203 - accuracy: 0.9956 - recall: 0.9956 - precision: 0.9956 - val_loss: 0.0208 - val_accuracy: 0.9954 - val_recall: 0.9954 - val_precision: 0.9954
Epoch 60/100
1955/1955 [==============================] - 10s 5ms/step - loss: 0.020

Epoch 92/100
1955/1955 [==============================] - 10s 5ms/step - loss: 0.0158 - accuracy: 0.9969 - recall: 0.9969 - precision: 0.9969 - val_loss: 0.0157 - val_accuracy: 0.9965 - val_recall: 0.9965 - val_precision: 0.9965
Epoch 93/100
1955/1955 [==============================] - 10s 5ms/step - loss: 0.0143 - accuracy: 0.9971 - recall: 0.9971 - precision: 0.9971 - val_loss: 0.0114 - val_accuracy: 0.9983 - val_recall: 0.9983 - val_precision: 0.9983
Epoch 94/100
1955/1955 [==============================] - 10s 5ms/step - loss: 0.0144 - accuracy: 0.9971 - recall: 0.9971 - precision: 0.9971 - val_loss: 0.0137 - val_accuracy: 0.9971 - val_recall: 0.9971 - val_precision: 0.9971
Epoch 95/100
1955/1955 [==============================] - 10s 5ms/step - loss: 0.0144 - accuracy: 0.9971 - recall: 0.9971 - precision: 0.9971 - val_loss: 0.0088 - val_accuracy: 0.9992 - val_recall: 0.9992 - val_precision: 0.9992
Epoch 96/100
1955/1955 [==============================] - 10s 5ms/step - loss: 0.014

In [ ]:
# plot the following:
# - Training loss
# - Validation loss
# - Training accuracy 
# - Validation accuracy
fig, ax = plt.subplots(1,2,figsize = (16,8))
ax[0].plot(history_reg.history['loss'],color='#EFAEA4',label = 'Training Loss')
ax[0].plot(history_reg.history['val_loss'],color='#B2D7D0',label = 'Validation Loss')
ax[1].plot(history_reg.history['accuracy'],color='#EFAEA4',label = 'Training Accuracy')
ax[1].plot(history_reg.history['val_accuracy'],color='#B2D7D0',label = 'Validation Accuracy')
ax[0].legend()
ax[1].legend()
ax[0].set_xlabel('Epochs')
ax[1].set_xlabel('Epochs');
ax[0].set_ylabel('Loss')
ax[1].set_ylabel('Accuracy %');

In [ ]:
# stats
y_pred_ohe = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_true_labels = np.argmax(y_test, axis=1)
print("*" * 80)
print("Classification report")
print("*" * 80)
print(classification_report(y_true_labels, y_pred_labels))